In [0]:
!pip install scikit-learn
!pip install numpy==1.16.1

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/')
!ls
os.chdir('My Drive/Computer_Vision_Masters/Wearable_Sensors_Code/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
'My Drive'


In [0]:
## Libraries
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics.classification import accuracy_score, recall_score, f1_score
import scipy.stats as st
import sys
import warnings
warnings.filterwarnings('ignore')
np.random.seed(12227)

In [0]:
import scipy.stats as st
from scipy.fftpack import fft, fftfreq 
from scipy.signal import argrelextrema
import operator

In [0]:
def DataPreparation(data_input_file):
  print('Proposed Model {}'.format(data_input_file))
  data = np.load(data_input_file)
  X = data['X']
  X = X[:, 0, :, :]
  Y = data['y']
  folds = data['folds']
  classes_number = Y.shape[1]
  print(classes_number)
  Y = np.argmax(Y, axis=1)
  return X,Y,folds,classes_number

In [0]:
def mean_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
    # print(sample[:,col].shape)
    average = np.mean(sample[:,col])
    feat.append(average)

  return feat

def max_ts(sample, Te=1.0):
   
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amax(sample[:,col])
      feat.append(average)

  return feat

def min_ts(sample, Te=1.0): 
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.amin(sample[:,col]) # min
      feat.append(average)

  return feat 

def skew_ts(sample, Te=1.0):    
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.skew(sample[:,col]) # skew
      feat.append(average)

  return feat

def kurtosis_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.kurtosis(sample[:,col]) # kurtosis 
      feat.append(average)

  return feat
 
def iqr_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = st.iqr(sample[:,col]) # interquartile rante
      feat.append(average)

  return feat

def mad_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.median(np.sort(abs(sample[:,col] - np.median(sample[:,col]))))
      feat.append(average)

  return feat

def area_ts(sample, Te=1.0):  
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col], dx=Te)
      feat.append(average)

  return feat                            
 

def sq_area_ts(sample, Te=1.0):
  feat = []
  for col in range(0,sample.shape[1]):
      average = np.trapz(sample[:,col] ** 2, dx=Te)
      feat.append(average)

  return feat
def energy_measure(sample):
  """Calculates energy measures"""
  feat = []
  for col in range(0,sample.shape[1]):

    em_x = np.mean(np.square(sample[:,col] ))
    feat.append(em_x)
  return feat  

  # feature.append(mean_ts)
  # feature.append(max_ts)
  # feature.append(min_ts)
  # feature.append(skew_ts)
  # feature.append(kurtosis_ts)
  # feature.append(iqr_ts)
  # feature.append(mad_ts)
  # feature.append(area_ts)
  # feature.append(sq_area_ts)



In [0]:
def stat_area_features(x, Te=1.0):

    mean_ts = np.mean(x, axis=1).reshape(-1,1) # mean
    max_ts = np.amax(x, axis=1).reshape(-1,1) # max
    min_ts = np.amin(x, axis=1).reshape(-1,1) # min
    std_ts = np.std(x, axis=1).reshape(-1,1) # std
    skew_ts = st.skew(x, axis=1).reshape(-1,1) # skew
    kurtosis_ts = st.kurtosis(x, axis=1).reshape(-1,1) # kurtosis 
    iqr_ts = st.iqr(x, axis=1).reshape(-1,1) # interquartile rante
    mad_ts = np.median(np.sort(abs(x - np.median(x, axis=1).reshape(-1,1)),
                               axis=1), axis=1).reshape(-1,1) # median absolute deviation
    area_ts = np.trapz(x, axis=1, dx=Te).reshape(-1,1) # area under curve
    sq_area_ts = np.trapz(x ** 2, axis=1, dx=Te).reshape(-1,1) # area under curve ** 2

    return np.concatenate((mean_ts,max_ts,min_ts,std_ts,skew_ts,kurtosis_ts,
                           iqr_ts,mad_ts,area_ts,sq_area_ts), axis=1)
def frequency_domain_features(x, Te=1.0):
  feat=[]

  # As the DFT coefficients and their corresponding frequencies are symetrical arrays
  # with respect to the middle of the array we need to know if the number of readings 
  # in x is even or odd to then split the arrays...
  if x.shape[1]%2 == 0:
      N = int(x.shape[1]/2)
  else:
      N = int(x.shape[1]/2) - 1
  xf = np.repeat(fftfreq(x.shape[1],d=Te)[:N].reshape(1,-1), x.shape[0], axis=0) # frequencies
  dft = np.abs(fft(x, axis=1))[:,:N] # DFT coefficients   
  
  # statistical and area features
  # dft_features = stat_area_features(dft, Te=1.0)
  # weighted mean frequency
  dft_weighted_mean_f = np.average(xf, axis=1, weights=dft).reshape(-1,1)
  # 5 first DFT coefficients 
  dft_first_coef = dft[:,:5]    
  # 5 first local maxima of DFT coefficients and their corresponding frequencies
  dft_max_coef = np.zeros((x.shape[0],5))
  dft_max_coef_f = np.zeros((x.shape[0],5))
  for row in range(x.shape[0]):
      # finds all local maximas indexes
      extrema_ind = argrelextrema(dft[row,:], np.greater, axis=0) 
      # makes a list of tuples (DFT_i, f_i) of all the local maxima
      # and keeps the 5 biggest...
      extrema_row = sorted([(dft[row,:][j],xf[row,j]) for j in extrema_ind[0]],
                            key=operator.itemgetter(0), reverse=True)[:5] 
      for i, ext in enumerate(extrema_row):
          dft_max_coef[row,i] = ext[0]
          dft_max_coef_f[row,i] = ext[1]  

  feat.append(np.concatenate((dft_weighted_mean_f,dft_first_coef, dft_max_coef,dft_max_coef_f), axis=1).tolist() )
  # print(len(feat))
  # print(feat)
  return feat

In [0]:
def A(sample):
    feat = []
    for col in range(0,sample.shape[1]):
        average = np.average(sample[:,col])
        feat.append(average)

    return feat

def SD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        std = np.std(sample[:, col])
        feat.append(std)

    return feat

def AAD(sample):
    feat = []
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        add = np.mean(np.absolute(data - np.mean(data)))
        feat.append(add)

    return feat

def ARA(sample):
    #Average Resultant Acceleration[1]:
    # Average of the square roots of the sum of the values of each axis squared √(xi^2 + yi^2+ zi^2) over the ED
    feat = []
    sum_square = 0
    sample = np.power(sample, 2)
    for col in range(0, sample.shape[1]):
        sum_square = sum_square + sample[:, col]

    sample = np.sqrt(sum_square)
    average = np.average(sample)
    feat.append(average)
    return feat

def TBP(sample):
    from scipy import signal
    feat = []
    sum_of_time = 0
    for col in range(0, sample.shape[1]):
        data = sample[:, col]
        peaks = signal.find_peaks_cwt(data, np.arange(1,4))

        feat.append(peaks.tolist())

    return feat

In [0]:
## Feature Extraction

def feature_extraction(X):
    # Extracts the features, as mentioned by Catal et al. 2015
    # Average - A,
    # Standard Deviation - SD,
    # Average Absolute Difference - AAD,
    # Average Resultant Acceleration - ARA(1),
    # Time Between Peaks - TBP
    X_tmp = []
    # min_feature=1000000000
    # cnt=0
    for sample in X:
        features = A(sample)
        features = np.hstack((features, A(sample)))
        features = np.hstack((features, SD(sample)))
        features = np.hstack((features, AAD(sample)))
        features = np.hstack((features, ARA(sample)))
        # features = np.hstack((features, TBP(sample)))
        features = np.hstack((features, mean_ts(sample)))
        features = np.hstack((features, max_ts(sample)))
        features = np.hstack((features, min_ts(sample)))
        features = np.hstack((features, kurtosis_ts(sample)))
        features = np.hstack((features, iqr_ts(sample)))
        features = np.hstack((features, skew_ts(sample)))
        features = np.hstack((features, area_ts(sample)))
        features = np.hstack((features, sq_area_ts(sample)))
        features = np.hstack((features, mad_ts(sample)))
        # print(TBP(sample))
        # TBP_=TBP(sample)
        # for l in TBP_:
        #   features = np.hstack((features,l))
          # print('l',len(l))

          # print(l)
        # frequency_domain_features_= frequency_domain_features(sample)
        # cnt=1
        # for list_ in frequency_domain_features_:
        #   if cnt==1:
        #     for l in list_:
        #       features = np.hstack((features,l))
        #   else:
        #    features = np.hstack((features,list_))
        #   cnt+=1
               
          
          
          
        # features = np.hstack((features, energy_measure(sample)))
        # print('skew',skew_ts(sample))
        # print('kurtosis_ts',kurtosis_ts(sample))
        # print('iqr_ts',iqr_ts(sample))
        # print('A',A(sample))
        # print(len(mad_ts(sample)))
        # print(len(skew_ts(sample)))
        # print(len(ARA(sample)))
        # print(len(A(sample)))
        # print(len(SD(sample)))
        X_tmp.append(features)
        # cnt+=1
        # # print(len(features))
        # min_feature=min(min_feature,len(features))

    # print(len(X_tmp))
    X = np.array(X_tmp)
    # print(X.shape)
    return X

In [11]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.2.0-rc3
/device:GPU:0
Num GPUs Available:  1


In [0]:
# def Model(num_classes):
#   layers = [
    
#     #  tf.keras.layers.BatchNormalization(),
#       # tf.keras.layers.Dense(256, activation = tf.nn.relu),
#       # tf.keras.layers.LSTM(64),

#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(128, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(64, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
#       tf.keras.layers.Dense(32, activation = tf.nn.relu),
#       # tf.keras.layers.BatchNormalization(),
      
#       tf.keras.layers.Dense(num_classes, activation = 'softmax')
#   ]
#   model = tf.keras.Sequential(layers)
#   return model

In [0]:
def Model(num_classes):
  layers = [
            # tf.keras.layers.Dense(256, activation = tf.nn.leaky_relu),
            # tf.keras.layers.Dropout(0.2),

      tf.keras.layers.Dense(128, activation =tf.nn.leaky_relu),
      # tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(64, activation =tf.nn.leaky_relu),
      # tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(32, activation = tf.nn.leaky_relu),
      # tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(num_classes, activation = 'softmax')
  ]
  model = tf.keras.Sequential(layers)
  return model

In [0]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split


In [0]:
from tensorflow.keras.callbacks import EarlyStopping
def TrainAndReportAccuracy(X,Y,folds,num_classes):
  avg_acc = []
  X_train, X_test, y_train, y_test=train_test_split( X, Y, test_size=0.33, random_state=42)
  X_train = feature_extraction(X_train)
  X_test = feature_extraction(X_test)
  # train_idx = folds[0][0]
  # test_idx = folds[0][1]
  # X_train = X[train_idx]
  # X_test = X[test_idx]
  # Total_Ytrain = Y[train_idx]
  # Total_Ytest = Y[test_idx]
  # Total_Xtrain = feature_extraction(X_train)
  # Total_Xtest = feature_extraction(X_test)
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA
  # for i in range(1, len(folds)):
  #         train_idx = folds[i][0]
  #         test_idx = folds[i][1]

  #         X_train = X[train_idx]
  #         X_test = X[test_idx]
  #         Y_train = Y[train_idx]
  #         Y_test = Y[test_idx]
  #         X_train = feature_extraction(X_train)
  #         X_test = feature_extraction(X_test)
  #         Total_Xtrain=np.append(Total_Xtrain, X_train,axis=0)
  #         Total_Ytrain=np.append(Total_Ytrain, Y_train)
  #         Total_Xtest=np.append(Total_Xtest, X_test,axis=0)
  #         Total_Ytest=np.append(Total_Ytest, Y_test)


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, y_train.shape)
  print(X_test.shape,y_test.shape)
  # size, rows = X_train.shape
  # X_train=X_train.reshape(X_train.shape[0],rows,1)
  # y_train=y_train.reshape(y_train.shape[0],num_classes)
  print(X_train.shape)
  # _,img_rows, img_cols,_ = X.shape

  model = Model(num_classes)
  es = EarlyStopping(monitor='sparse_categorical_accuracy', mode='max', verbose=1, patience=100)
  

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, y_train, batch_size=64, epochs=250)
  #,callbacks=[es]
  _,acc=model.evaluate(X_test,y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))
  

In [0]:
def Run(data_input_file):
  X,Y,folds,num_classes=DataPreparation(data_input_file)
  TrainAndReportAccuracy(X,Y,folds,num_classes)

In [0]:
def TrainAndReportAccuracyOpportunity(X_train,Y_train,X_test,Y_test,num_classes):
  avg_acc = []
  from sklearn.preprocessing import StandardScaler
  from sklearn.decomposition import PCA


  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  pca = PCA(.95)
  pca.fit(X_train)
  X_train = pca.transform(X_train)
  X_test = pca.transform(X_test)
  print(X_train.shape, Y_train.shape)
  print(X_test.shape,Y_test.shape)

  model = Model(num_classes)
  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=[tf.keras.metrics.sparse_categorical_accuracy])
  history = model.fit(X_train, Y_train, batch_size=32, epochs=250)
  _,acc=model.evaluate(X_test,Y_test)
  avg_acc.append(acc)
  print('________________________________________________________________')
  print('Mean Accuracy[{:.4f}] '.format(np.mean(avg_acc)))

In [0]:
def OpportunityDataSetAnalysis():
  X_train=np.load('data/Opportunity/Opportunity_train_X.npz')['arr_0']
  X_test=np.load('data/Opportunity/Opportunity_test_X.npz')['arr_0']
  Y_train=np.load('data/Opportunity/Opportunity_train_y.npz')['arr_0']
  Y_test=np.load('data/Opportunity/Opportunity_test_Y.npz')['arr_0']
  classes_number = Y_train.shape[1]
  Y_train = np.argmax(Y_train, axis=1)
  Y_test = np.argmax(Y_test, axis=1)
  return X_train,Y_train,X_test,Y_test,classes_number

In [0]:
X_train,Y_train,X_test,Y_test,num_classes=OpportunityDataSetAnalysis()
  

In [0]:
X_train = feature_extraction(X_train)
X_test = feature_extraction(X_test)

In [29]:
TrainAndReportAccuracyOpportunity(X_train,Y_train,X_test,Y_test,num_classes)

(46495, 313) (46495,)
(9894, 313) (9894,)
Epoch 1/250
1453/1453 [==============================] - 5s 3ms/step - loss: 0.3226 - sparse_categorical_accuracy: 0.8818
Epoch 2/250
1453/1453 [==============================] - 5s 4ms/step - loss: 0.1996 - sparse_categorical_accuracy: 0.9253
Epoch 3/250
1453/1453 [==============================] - 5s 3ms/step - loss: 0.1586 - sparse_categorical_accuracy: 0.9403
Epoch 4/250
1453/1453 [==============================] - 5s 3ms/step - loss: 0.1329 - sparse_categorical_accuracy: 0.9491
Epoch 5/250
1453/1453 [==============================] - 5s 3ms/step - loss: 0.1113 - sparse_categorical_accuracy: 0.9579
Epoch 6/250
1453/1453 [==============================] - 4s 3ms/step - loss: 0.0949 - sparse_categorical_accuracy: 0.9636
Epoch 7/250
1453/1453 [==============================] - 4s 3ms/step - loss: 0.0782 - sparse_categorical_accuracy: 0.9711
Epoch 8/250
1453/1453 [==============================] - 5s 3ms/step - loss: 0.0666 - sparse_categorical

In [20]:
RunOpportunity()

(46495, 313) (46495,)
(9894, 313) (9894,)
Epoch 1/250
727/727 [==============================] - 2s 3ms/step - loss: 0.3372 - sparse_categorical_accuracy: 0.8736
Epoch 2/250
727/727 [==============================] - 2s 3ms/step - loss: 0.2017 - sparse_categorical_accuracy: 0.9254
Epoch 3/250
727/727 [==============================] - 2s 3ms/step - loss: 0.1597 - sparse_categorical_accuracy: 0.9406
Epoch 4/250
727/727 [==============================] - 2s 3ms/step - loss: 0.1302 - sparse_categorical_accuracy: 0.9521
Epoch 5/250
727/727 [==============================] - 2s 3ms/step - loss: 0.1092 - sparse_categorical_accuracy: 0.9593
Epoch 6/250
727/727 [==============================] - 2s 3ms/step - loss: 0.0895 - sparse_categorical_accuracy: 0.9665
Epoch 7/250
727/727 [==============================] - 2s 3ms/step - loss: 0.0742 - sparse_categorical_accuracy: 0.9720
Epoch 8/250
727/727 [==============================] - 2s 3ms/step - loss: 0.0613 - sparse_categorical_accuracy: 0.977

In [0]:
Run('data/LOSO/MHEALTH.npz')

Proposed Model data/LOSO/MHEALTH.npz
12
(1711, 56) (1711,)
(844, 56) (844,)
(1711, 56)
Epoch 1/250
27/27 [==============================] - 0s 2ms/step - loss: 1.5072 - sparse_categorical_accuracy: 0.5535
Epoch 2/250
27/27 [==============================] - 0s 2ms/step - loss: 0.3534 - sparse_categorical_accuracy: 0.9702
Epoch 3/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0818 - sparse_categorical_accuracy: 0.9953
Epoch 4/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0338 - sparse_categorical_accuracy: 0.9994
Epoch 5/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0190 - sparse_categorical_accuracy: 0.9988
Epoch 6/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0119 - sparse_categorical_accuracy: 1.0000
Epoch 7/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0083 - sparse_categorical_accuracy: 1.0000
Epoch 8/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0065 - spars

In [0]:
Run('data/LOTO/MHEALTH.npz')

Proposed Model data/LOTO/MHEALTH.npz
12
(1711, 56) (1711,)
(844, 56) (844,)
(1711, 56)
Epoch 1/250
27/27 [==============================] - 0s 2ms/step - loss: 1.5749 - sparse_categorical_accuracy: 0.5295
Epoch 2/250
27/27 [==============================] - 0s 2ms/step - loss: 0.4132 - sparse_categorical_accuracy: 0.9509
Epoch 3/250
27/27 [==============================] - 0s 2ms/step - loss: 0.1103 - sparse_categorical_accuracy: 0.9953
Epoch 4/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0448 - sparse_categorical_accuracy: 0.9994
Epoch 5/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0257 - sparse_categorical_accuracy: 0.9977
Epoch 6/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0159 - sparse_categorical_accuracy: 1.0000
Epoch 7/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0113 - sparse_categorical_accuracy: 1.0000
Epoch 8/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0083 - spars

In [0]:
Run('data/FNOW/MHEALTH.npz')

Proposed Model data/FNOW/MHEALTH.npz
12
(894, 55) (894,)
(441, 55) (441,)
(894, 55)
Epoch 1/250
14/14 [==============================] - 0s 2ms/step - loss: 2.3184 - sparse_categorical_accuracy: 0.2394
Epoch 2/250
14/14 [==============================] - 0s 2ms/step - loss: 1.0805 - sparse_categorical_accuracy: 0.7562
Epoch 3/250
14/14 [==============================] - 0s 2ms/step - loss: 0.5561 - sparse_categorical_accuracy: 0.9195
Epoch 4/250
14/14 [==============================] - 0s 2ms/step - loss: 0.2613 - sparse_categorical_accuracy: 0.9843
Epoch 5/250
14/14 [==============================] - 0s 2ms/step - loss: 0.1273 - sparse_categorical_accuracy: 0.9922
Epoch 6/250
14/14 [==============================] - 0s 2ms/step - loss: 0.0702 - sparse_categorical_accuracy: 0.9978
Epoch 7/250
14/14 [==============================] - 0s 2ms/step - loss: 0.0448 - sparse_categorical_accuracy: 0.9989
Epoch 8/250
14/14 [==============================] - 0s 2ms/step - loss: 0.0313 - sparse_c

In [0]:
Run('data/SNOW/MHEALTH.npz')

Proposed Model data/SNOW/MHEALTH.npz
12
(1711, 56) (1711,)
(844, 56) (844,)
(1711, 56)
Epoch 1/250
27/27 [==============================] - 0s 2ms/step - loss: 1.5815 - sparse_categorical_accuracy: 0.5213
Epoch 2/250
27/27 [==============================] - 0s 2ms/step - loss: 0.3558 - sparse_categorical_accuracy: 0.9515
Epoch 3/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0895 - sparse_categorical_accuracy: 0.9965
Epoch 4/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0365 - sparse_categorical_accuracy: 0.9977
Epoch 5/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0211 - sparse_categorical_accuracy: 0.9988
Epoch 6/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0143 - sparse_categorical_accuracy: 1.0000
Epoch 7/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0101 - sparse_categorical_accuracy: 1.0000
Epoch 8/250
27/27 [==============================] - 0s 2ms/step - loss: 0.0074 - spars

In [0]:
Run('data/LOTO/USCHAD.npz')

Proposed Model data/LOTO/USCHAD.npz
12
(6582, 19) (6582,)
(3242, 19) (3242,)
(6582, 19)
Epoch 1/250
103/103 [==============================] - 0s 2ms/step - loss: 1.0901 - sparse_categorical_accuracy: 0.6612
Epoch 2/250
103/103 [==============================] - 0s 2ms/step - loss: 0.4718 - sparse_categorical_accuracy: 0.8294
Epoch 3/250
103/103 [==============================] - 0s 2ms/step - loss: 0.3760 - sparse_categorical_accuracy: 0.8593
Epoch 4/250
103/103 [==============================] - 0s 2ms/step - loss: 0.3242 - sparse_categorical_accuracy: 0.8774
Epoch 5/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2909 - sparse_categorical_accuracy: 0.8914
Epoch 6/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2641 - sparse_categorical_accuracy: 0.8988
Epoch 7/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2377 - sparse_categorical_accuracy: 0.9076
Epoch 8/250
103/103 [==============================] - 0s 2ms/step - los

In [0]:
Run('data/LOSO/USCHAD.npz')

Proposed Model data/LOSO/USCHAD.npz
12
(6582, 19) (6582,)
(3242, 19) (3242,)
(6582, 19)
Epoch 1/250
103/103 [==============================] - 0s 2ms/step - loss: 1.0507 - sparse_categorical_accuracy: 0.6820
Epoch 2/250
103/103 [==============================] - 0s 2ms/step - loss: 0.4688 - sparse_categorical_accuracy: 0.8262
Epoch 3/250
103/103 [==============================] - 0s 2ms/step - loss: 0.3803 - sparse_categorical_accuracy: 0.8589
Epoch 4/250
103/103 [==============================] - 0s 2ms/step - loss: 0.3235 - sparse_categorical_accuracy: 0.8780
Epoch 5/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2859 - sparse_categorical_accuracy: 0.8924
Epoch 6/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2539 - sparse_categorical_accuracy: 0.8999
Epoch 7/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2373 - sparse_categorical_accuracy: 0.9053
Epoch 8/250
103/103 [==============================] - 0s 2ms/step - los

In [0]:
Run('data/FNOW/USCHAD.npz')

Proposed Model data/FNOW/USCHAD.npz
12
(3375, 19) (3375,)
(1663, 19) (1663,)
(3375, 19)
Epoch 1/250
53/53 [==============================] - 0s 2ms/step - loss: 1.5640 - sparse_categorical_accuracy: 0.5440
Epoch 2/250
53/53 [==============================] - 0s 2ms/step - loss: 0.6541 - sparse_categorical_accuracy: 0.7778
Epoch 3/250
53/53 [==============================] - 0s 2ms/step - loss: 0.4626 - sparse_categorical_accuracy: 0.8373
Epoch 4/250
53/53 [==============================] - 0s 2ms/step - loss: 0.3930 - sparse_categorical_accuracy: 0.8498
Epoch 5/250
53/53 [==============================] - 0s 2ms/step - loss: 0.3438 - sparse_categorical_accuracy: 0.8747
Epoch 6/250
53/53 [==============================] - 0s 2ms/step - loss: 0.3142 - sparse_categorical_accuracy: 0.8853
Epoch 7/250
53/53 [==============================] - 0s 2ms/step - loss: 0.2918 - sparse_categorical_accuracy: 0.8868
Epoch 8/250
53/53 [==============================] - 0s 2ms/step - loss: 0.2734 - spar

In [0]:
Run('data/SNOW/USCHAD.npz')

Proposed Model data/SNOW/USCHAD.npz
12
(6582, 19) (6582,)
(3242, 19) (3242,)
(6582, 19)
Epoch 1/250
103/103 [==============================] - 0s 2ms/step - loss: 1.0049 - sparse_categorical_accuracy: 0.6714
Epoch 2/250
103/103 [==============================] - 0s 2ms/step - loss: 0.4463 - sparse_categorical_accuracy: 0.8301
Epoch 3/250
103/103 [==============================] - 0s 2ms/step - loss: 0.3535 - sparse_categorical_accuracy: 0.8671
Epoch 4/250
103/103 [==============================] - 0s 2ms/step - loss: 0.3075 - sparse_categorical_accuracy: 0.8854
Epoch 5/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2684 - sparse_categorical_accuracy: 0.9009
Epoch 6/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2468 - sparse_categorical_accuracy: 0.9057
Epoch 7/250
103/103 [==============================] - 0s 2ms/step - loss: 0.2256 - sparse_categorical_accuracy: 0.9129
Epoch 8/250
103/103 [==============================] - 0s 2ms/step - los

In [0]:
Run('data/LOTO/WHARF.npz')

Proposed Model data/LOTO/WHARF.npz
12
(2593, 11) (2593,)
(1278, 11) (1278,)
(2593, 11)
Epoch 1/250
41/41 [==============================] - 0s 2ms/step - loss: 1.7000 - sparse_categorical_accuracy: 0.4458
Epoch 2/250
41/41 [==============================] - 0s 2ms/step - loss: 1.1793 - sparse_categorical_accuracy: 0.6128
Epoch 3/250
41/41 [==============================] - 0s 2ms/step - loss: 0.9991 - sparse_categorical_accuracy: 0.6479
Epoch 4/250
41/41 [==============================] - 0s 2ms/step - loss: 0.9122 - sparse_categorical_accuracy: 0.6753
Epoch 5/250
41/41 [==============================] - 0s 2ms/step - loss: 0.8534 - sparse_categorical_accuracy: 0.6938
Epoch 6/250
41/41 [==============================] - 0s 2ms/step - loss: 0.8063 - sparse_categorical_accuracy: 0.7042
Epoch 7/250
41/41 [==============================] - 0s 2ms/step - loss: 0.7739 - sparse_categorical_accuracy: 0.7177
Epoch 8/250
41/41 [==============================] - 0s 2ms/step - loss: 0.7503 - spars

In [0]:
Run('data/LOSO/WHARF.npz')

Proposed Model data/LOSO/WHARF.npz
12
(2593, 11) (2593,)
(1278, 11) (1278,)
(2593, 11)
Epoch 1/250
41/41 [==============================] - 0s 2ms/step - loss: 1.7911 - sparse_categorical_accuracy: 0.4292
Epoch 2/250
41/41 [==============================] - 0s 2ms/step - loss: 1.1508 - sparse_categorical_accuracy: 0.6024
Epoch 3/250
41/41 [==============================] - 0s 2ms/step - loss: 0.9545 - sparse_categorical_accuracy: 0.6683
Epoch 4/250
41/41 [==============================] - 0s 2ms/step - loss: 0.8726 - sparse_categorical_accuracy: 0.6888
Epoch 5/250
41/41 [==============================] - 0s 2ms/step - loss: 0.8218 - sparse_categorical_accuracy: 0.7011
Epoch 6/250
41/41 [==============================] - 0s 2ms/step - loss: 0.7854 - sparse_categorical_accuracy: 0.7050
Epoch 7/250
41/41 [==============================] - 0s 2ms/step - loss: 0.7554 - sparse_categorical_accuracy: 0.7208
Epoch 8/250
41/41 [==============================] - 0s 2ms/step - loss: 0.7290 - spars

In [0]:
Run('data/SNOW/WHARF.npz')

Proposed Model data/SNOW/WHARF.npz
12
(2599, 11) (2599,)
(1281, 11) (1281,)
(2599, 11)
Epoch 1/250
41/41 [==============================] - 0s 2ms/step - loss: 1.7124 - sparse_categorical_accuracy: 0.4656
Epoch 2/250
41/41 [==============================] - 0s 2ms/step - loss: 1.1939 - sparse_categorical_accuracy: 0.6010
Epoch 3/250
41/41 [==============================] - 0s 2ms/step - loss: 1.0307 - sparse_categorical_accuracy: 0.6391
Epoch 4/250
41/41 [==============================] - 0s 2ms/step - loss: 0.9254 - sparse_categorical_accuracy: 0.6645
Epoch 5/250
41/41 [==============================] - 0s 2ms/step - loss: 0.8573 - sparse_categorical_accuracy: 0.6864
Epoch 6/250
41/41 [==============================] - 0s 2ms/step - loss: 0.8081 - sparse_categorical_accuracy: 0.7064
Epoch 7/250
41/41 [==============================] - 0s 2ms/step - loss: 0.7711 - sparse_categorical_accuracy: 0.7168
Epoch 8/250
41/41 [==============================] - 0s 2ms/step - loss: 0.7416 - spars

In [0]:
Run('data/FNOW/WHARF.npz')

Proposed Model data/FNOW/WHARF.npz
12
(1437, 11) (1437,)
(709, 11) (709,)
(1437, 11)
Epoch 1/250
23/23 [==============================] - 0s 2ms/step - loss: 2.0996 - sparse_categorical_accuracy: 0.3716
Epoch 2/250
23/23 [==============================] - 0s 2ms/step - loss: 1.4722 - sparse_categorical_accuracy: 0.5219
Epoch 3/250
23/23 [==============================] - 0s 2ms/step - loss: 1.2238 - sparse_categorical_accuracy: 0.5943
Epoch 4/250
23/23 [==============================] - 0s 2ms/step - loss: 1.1017 - sparse_categorical_accuracy: 0.6180
Epoch 5/250
23/23 [==============================] - 0s 2ms/step - loss: 1.0126 - sparse_categorical_accuracy: 0.6402
Epoch 6/250
23/23 [==============================] - 0s 2ms/step - loss: 0.9444 - sparse_categorical_accuracy: 0.6604
Epoch 7/250
23/23 [==============================] - 0s 2ms/step - loss: 0.9052 - sparse_categorical_accuracy: 0.6701
Epoch 8/250
23/23 [==============================] - 0s 2ms/step - loss: 0.8675 - sparse_

In [0]:
Run('data/LOTO/WISDM.npz')

Proposed Model data/LOTO/WISDM.npz
6
(13966, 11) (13966,)
(6880, 11) (6880,)
(13966, 11)
Epoch 1/250
219/219 [==============================] - 0s 2ms/step - loss: 0.5313 - sparse_categorical_accuracy: 0.8241
Epoch 2/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2905 - sparse_categorical_accuracy: 0.9009
Epoch 3/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2362 - sparse_categorical_accuracy: 0.9180
Epoch 4/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2064 - sparse_categorical_accuracy: 0.9275
Epoch 5/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1809 - sparse_categorical_accuracy: 0.9365
Epoch 6/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1681 - sparse_categorical_accuracy: 0.9430
Epoch 7/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1541 - sparse_categorical_accuracy: 0.9480
Epoch 8/250
219/219 [==============================] - 0s 2ms/step - lo

In [0]:
Run('data/LOSO/WISDM.npz')

Proposed Model data/LOSO/WISDM.npz
6
(13966, 11) (13966,)
(6880, 11) (6880,)
(13966, 11)
Epoch 1/250
219/219 [==============================] - 0s 2ms/step - loss: 0.5782 - sparse_categorical_accuracy: 0.8072
Epoch 2/250
219/219 [==============================] - 0s 2ms/step - loss: 0.3122 - sparse_categorical_accuracy: 0.8893
Epoch 3/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2557 - sparse_categorical_accuracy: 0.9094
Epoch 4/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2222 - sparse_categorical_accuracy: 0.9217
Epoch 5/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2004 - sparse_categorical_accuracy: 0.9288
Epoch 6/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1836 - sparse_categorical_accuracy: 0.9375
Epoch 7/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1686 - sparse_categorical_accuracy: 0.9425
Epoch 8/250
219/219 [==============================] - 0s 2ms/step - lo

In [0]:
Run('data/SNOW/WISDM.npz')

Proposed Model data/SNOW/WISDM.npz
6
(13966, 11) (13966,)
(6880, 11) (6880,)
(13966, 11)
Epoch 1/250
219/219 [==============================] - 0s 2ms/step - loss: 0.5361 - sparse_categorical_accuracy: 0.8252
Epoch 2/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2964 - sparse_categorical_accuracy: 0.8998
Epoch 3/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2401 - sparse_categorical_accuracy: 0.9162
Epoch 4/250
219/219 [==============================] - 0s 2ms/step - loss: 0.2094 - sparse_categorical_accuracy: 0.9255
Epoch 5/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1895 - sparse_categorical_accuracy: 0.9335
Epoch 6/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1690 - sparse_categorical_accuracy: 0.9401
Epoch 7/250
219/219 [==============================] - 0s 2ms/step - loss: 0.1570 - sparse_categorical_accuracy: 0.9471
Epoch 8/250
219/219 [==============================] - 0s 2ms/step - lo

In [0]:
Run('data/FNOW/WISDM.npz')

Proposed Model data/FNOW/WISDM.npz
6
(7045, 11) (7045,)
(3471, 11) (3471,)
(7045, 11)
Epoch 1/250
111/111 [==============================] - 0s 2ms/step - loss: 0.6832 - sparse_categorical_accuracy: 0.7824
Epoch 2/250
111/111 [==============================] - 0s 2ms/step - loss: 0.3811 - sparse_categorical_accuracy: 0.8690
Epoch 3/250
111/111 [==============================] - 0s 2ms/step - loss: 0.3244 - sparse_categorical_accuracy: 0.8893
Epoch 4/250
111/111 [==============================] - 0s 2ms/step - loss: 0.2845 - sparse_categorical_accuracy: 0.9001
Epoch 5/250
111/111 [==============================] - 0s 2ms/step - loss: 0.2586 - sparse_categorical_accuracy: 0.9094
Epoch 6/250
111/111 [==============================] - 0s 2ms/step - loss: 0.2328 - sparse_categorical_accuracy: 0.9171
Epoch 7/250
111/111 [==============================] - 0s 2ms/step - loss: 0.2126 - sparse_categorical_accuracy: 0.9256
Epoch 8/250
111/111 [==============================] - 0s 2ms/step - loss:

In [0]:
Run('data/LOTO/UTD-MHAD2_1s.npz')

Proposed Model data/LOTO/UTD-MHAD2_1s.npz
6
(761, 19) (761,)
(376, 19) (376,)
(761, 19)
Epoch 1/250
12/12 [==============================] - 0s 2ms/step - loss: 1.5471 - sparse_categorical_accuracy: 0.4008
Epoch 2/250
12/12 [==============================] - 0s 2ms/step - loss: 1.1358 - sparse_categorical_accuracy: 0.5650
Epoch 3/250
12/12 [==============================] - 0s 2ms/step - loss: 0.9130 - sparse_categorical_accuracy: 0.6491
Epoch 4/250
12/12 [==============================] - 0s 2ms/step - loss: 0.7492 - sparse_categorical_accuracy: 0.7477
Epoch 5/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6178 - sparse_categorical_accuracy: 0.7871
Epoch 6/250
12/12 [==============================] - 0s 2ms/step - loss: 0.5308 - sparse_categorical_accuracy: 0.8160
Epoch 7/250
12/12 [==============================] - 0s 2ms/step - loss: 0.4752 - sparse_categorical_accuracy: 0.8384
Epoch 8/250
12/12 [==============================] - 0s 2ms/step - loss: 0.4232 - spar

In [0]:
Run('data/LOSO/UTD-MHAD2_1s.npz')

Proposed Model data/LOSO/UTD-MHAD2_1s.npz
6
(761, 19) (761,)
(376, 19) (376,)
(761, 19)
Epoch 1/250
12/12 [==============================] - 0s 2ms/step - loss: 1.4485 - sparse_categorical_accuracy: 0.4731
Epoch 2/250
12/12 [==============================] - 0s 2ms/step - loss: 1.0406 - sparse_categorical_accuracy: 0.6518
Epoch 3/250
12/12 [==============================] - 0s 2ms/step - loss: 0.8200 - sparse_categorical_accuracy: 0.7083
Epoch 4/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6827 - sparse_categorical_accuracy: 0.7464
Epoch 5/250
12/12 [==============================] - 0s 2ms/step - loss: 0.5865 - sparse_categorical_accuracy: 0.7911
Epoch 6/250
12/12 [==============================] - 0s 2ms/step - loss: 0.5124 - sparse_categorical_accuracy: 0.8265
Epoch 7/250
12/12 [==============================] - 0s 2ms/step - loss: 0.4549 - sparse_categorical_accuracy: 0.8476
Epoch 8/250
12/12 [==============================] - 0s 2ms/step - loss: 0.4212 - spar

In [0]:
Run('data/FNOW/UTD-MHAD2_1s.npz')

Proposed Model data/FNOW/UTD-MHAD2_1s.npz
6
(412, 19) (412,)
(204, 19) (204,)
(412, 19)
Epoch 1/250
7/7 [==============================] - 0s 2ms/step - loss: 1.8312 - sparse_categorical_accuracy: 0.1772
Epoch 2/250
7/7 [==============================] - 0s 2ms/step - loss: 1.3612 - sparse_categorical_accuracy: 0.4539
Epoch 3/250
7/7 [==============================] - 0s 2ms/step - loss: 1.1121 - sparse_categorical_accuracy: 0.5995
Epoch 4/250
7/7 [==============================] - 0s 2ms/step - loss: 0.9808 - sparse_categorical_accuracy: 0.6602
Epoch 5/250
7/7 [==============================] - 0s 2ms/step - loss: 0.8537 - sparse_categorical_accuracy: 0.7039
Epoch 6/250
7/7 [==============================] - 0s 2ms/step - loss: 0.7564 - sparse_categorical_accuracy: 0.7379
Epoch 7/250
7/7 [==============================] - 0s 2ms/step - loss: 0.6903 - sparse_categorical_accuracy: 0.7621
Epoch 8/250
7/7 [==============================] - 0s 2ms/step - loss: 0.6202 - sparse_categorical_a

In [0]:
Run('data/SNOW/UTD-MHAD2_1s.npz')

Proposed Model data/SNOW/UTD-MHAD2_1s.npz
6
(761, 19) (761,)
(376, 19) (376,)
(761, 19)
Epoch 1/250
12/12 [==============================] - 0s 2ms/step - loss: 1.5347 - sparse_categorical_accuracy: 0.3968
Epoch 2/250
12/12 [==============================] - 0s 2ms/step - loss: 0.9996 - sparse_categorical_accuracy: 0.6623
Epoch 3/250
12/12 [==============================] - 0s 2ms/step - loss: 0.7747 - sparse_categorical_accuracy: 0.7267
Epoch 4/250
12/12 [==============================] - 0s 2ms/step - loss: 0.6465 - sparse_categorical_accuracy: 0.7714
Epoch 5/250
12/12 [==============================] - 0s 2ms/step - loss: 0.5689 - sparse_categorical_accuracy: 0.7819
Epoch 6/250
12/12 [==============================] - 0s 2ms/step - loss: 0.5039 - sparse_categorical_accuracy: 0.8173
Epoch 7/250
12/12 [==============================] - 0s 2ms/step - loss: 0.4664 - sparse_categorical_accuracy: 0.8357
Epoch 8/250
12/12 [==============================] - 0s 2ms/step - loss: 0.4252 - spar

In [0]:
Run('data/LOTO/UTD-MHAD1_1s.npz')

Proposed Model data/LOTO/UTD-MHAD1_1s.npz
21
(2526, 23) (2526,)
(1245, 23) (1245,)
(2526, 23)
Epoch 1/250
40/40 [==============================] - 0s 2ms/step - loss: 2.7823 - sparse_categorical_accuracy: 0.1797
Epoch 2/250
40/40 [==============================] - 0s 2ms/step - loss: 2.2267 - sparse_categorical_accuracy: 0.3413
Epoch 3/250
40/40 [==============================] - 0s 2ms/step - loss: 1.8579 - sparse_categorical_accuracy: 0.4327
Epoch 4/250
40/40 [==============================] - 0s 2ms/step - loss: 1.6383 - sparse_categorical_accuracy: 0.4929
Epoch 5/250
40/40 [==============================] - 0s 2ms/step - loss: 1.4991 - sparse_categorical_accuracy: 0.5360
Epoch 6/250
40/40 [==============================] - 0s 2ms/step - loss: 1.3972 - sparse_categorical_accuracy: 0.5610
Epoch 7/250
40/40 [==============================] - 0s 2ms/step - loss: 1.3105 - sparse_categorical_accuracy: 0.5871
Epoch 8/250
40/40 [==============================] - 0s 2ms/step - loss: 1.2381 

In [0]:
Run('data/LOSO/UTD-MHAD1_1s.npz')

Proposed Model data/LOSO/UTD-MHAD1_1s.npz
21
(2526, 23) (2526,)
(1245, 23) (1245,)
(2526, 23)
Epoch 1/250
40/40 [==============================] - 0s 2ms/step - loss: 2.7274 - sparse_categorical_accuracy: 0.1908
Epoch 2/250
40/40 [==============================] - 0s 2ms/step - loss: 2.1843 - sparse_categorical_accuracy: 0.3698
Epoch 3/250
40/40 [==============================] - 0s 2ms/step - loss: 1.8279 - sparse_categorical_accuracy: 0.4470
Epoch 4/250
40/40 [==============================] - 0s 2ms/step - loss: 1.6113 - sparse_categorical_accuracy: 0.4929
Epoch 5/250
40/40 [==============================] - 0s 2ms/step - loss: 1.4649 - sparse_categorical_accuracy: 0.5439
Epoch 6/250
40/40 [==============================] - 0s 2ms/step - loss: 1.3727 - sparse_categorical_accuracy: 0.5622
Epoch 7/250
40/40 [==============================] - 0s 2ms/step - loss: 1.2755 - sparse_categorical_accuracy: 0.5930
Epoch 8/250
40/40 [==============================] - 0s 2ms/step - loss: 1.2150 

In [0]:
Run('data/SNOW/UTD-MHAD1_1s.npz')

Proposed Model data/SNOW/UTD-MHAD1_1s.npz
21
(2526, 23) (2526,)
(1245, 23) (1245,)
(2526, 23)
Epoch 1/250
40/40 [==============================] - 0s 2ms/step - loss: 2.8202 - sparse_categorical_accuracy: 0.1477
Epoch 2/250
40/40 [==============================] - 0s 2ms/step - loss: 2.2929 - sparse_categorical_accuracy: 0.3298
Epoch 3/250
40/40 [==============================] - 0s 2ms/step - loss: 1.9032 - sparse_categorical_accuracy: 0.4446
Epoch 4/250
40/40 [==============================] - 0s 2ms/step - loss: 1.6489 - sparse_categorical_accuracy: 0.5067
Epoch 5/250
40/40 [==============================] - 0s 2ms/step - loss: 1.4844 - sparse_categorical_accuracy: 0.5443
Epoch 6/250
40/40 [==============================] - 0s 2ms/step - loss: 1.3768 - sparse_categorical_accuracy: 0.5780
Epoch 7/250
40/40 [==============================] - 0s 2ms/step - loss: 1.3017 - sparse_categorical_accuracy: 0.5887
Epoch 8/250
40/40 [==============================] - 0s 2ms/step - loss: 1.2264 

In [0]:
Run('data/FNOW/UTD-MHAD1_1s.npz')

Proposed Model data/FNOW/UTD-MHAD1_1s.npz
21
(1372, 22) (1372,)
(676, 22) (676,)
(1372, 22)
Epoch 1/250
22/22 [==============================] - 0s 2ms/step - loss: 2.9196 - sparse_categorical_accuracy: 0.1327
Epoch 2/250
22/22 [==============================] - 0s 2ms/step - loss: 2.5018 - sparse_categorical_accuracy: 0.2784
Epoch 3/250
22/22 [==============================] - 0s 2ms/step - loss: 2.1915 - sparse_categorical_accuracy: 0.3615
Epoch 4/250
22/22 [==============================] - 0s 2ms/step - loss: 1.9268 - sparse_categorical_accuracy: 0.4271
Epoch 5/250
22/22 [==============================] - 0s 2ms/step - loss: 1.7423 - sparse_categorical_accuracy: 0.4759
Epoch 6/250
22/22 [==============================] - 0s 2ms/step - loss: 1.6079 - sparse_categorical_accuracy: 0.5197
Epoch 7/250
22/22 [==============================] - 0s 2ms/step - loss: 1.4902 - sparse_categorical_accuracy: 0.5372
Epoch 8/250
22/22 [==============================] - 0s 2ms/step - loss: 1.3872 - 